# --- `Day 4`: Giant Squid ---

In [227]:
import aocd
import re
import operator
from collections import Counter, defaultdict, deque
from itertools import combinations
from functools import reduce, lru_cache

def prod(iterable):
    return reduce(operator.mul, iterable, 1)

def count(iterable, predicate = bool):
    return sum([1 for item in iterable if predicate(item)])

def first(iterable, default = None):
    return next(iter(iterable), default)

def lmap(func, *iterables):
    return list(map(func, *iterables))

def ints(s):
    return lmap(int, re.findall(r"-?\d+", s))

def words(s):
    return re.findall(r"[a-zA-Z]+", s)

def list_diff(x):
    return [b - a for a, b in zip(x, x[1:])]

def binary_to_int(lst):
    return int("".join(str(i) for i in lst), 2)

def get_column(lst, index):
    return [x[index] for x in lst]

In [228]:
def parse_line(line): 
    return [ints(x) for x in line.splitlines()]
    
def parse_input(input):
    return list(map(parse_line, input.split("\n\n")))

In [ ]:
final_input = parse_input(aocd.get_data(day=4, year=2021))
print(final_input[:5])

In [230]:
test_input = parse_input('''\
7,4,9,5,11,17,23,2,0,14,21,24,10,16,13,6,15,25,12,22,18,20,8,19,3,26,1

22 13 17 11  0
 8  2 23  4 24
21  9 14 16  7
 6 10  3 18  5
 1 12 20 15 19

 3 15  0  2 22
 9 18 13 17  5
19  8  7 25 23
20 11 10 24  4
14 21 16 12  6

14 21 17 24  4
10 16 15  9 19
18  8 23 26 20
22 11 13  6  5
 2  0 12  3  7
''')

print(test_input)

[[[7, 4, 9, 5, 11, 17, 23, 2, 0, 14, 21, 24, 10, 16, 13, 6, 15, 25, 12, 22, 18, 20, 8, 19, 3, 26, 1]], [[22, 13, 17, 11, 0], [8, 2, 23, 4, 24], [21, 9, 14, 16, 7], [6, 10, 3, 18, 5], [1, 12, 20, 15, 19]], [[3, 15, 0, 2, 22], [9, 18, 13, 17, 5], [19, 8, 7, 25, 23], [20, 11, 10, 24, 4], [14, 21, 16, 12, 6]], [[14, 21, 17, 24, 4], [10, 16, 15, 9, 19], [18, 8, 23, 26, 20], [22, 11, 13, 6, 5], [2, 0, 12, 3, 7]]]


### Helpers

In [231]:
def checkWin(lst):
    return all(x == '*' for x in lst)

assert checkWin([1,2,3,4,5]) == False
assert checkWin([1,2,3,'*',5]) == False
assert checkWin([1,'*','*',4,'*']) == False
assert checkWin(['*','*','*','*','*']) == True

In [232]:
def checkColumns(board):
    size = len(board[0])
    return any(checkWin(get_column(board, x)) for x in range(size))

assert checkColumns([[1, 2, 3], [4, 5, 6], [7, 8, 9]]) == False
assert checkColumns([['*', 2, 3], ['*', 5, 6], [7, 8, 9]]) == False
assert checkColumns([['*', 2, 3], ['*', 5, 6], ['*', 8, 9]]) == True
assert checkColumns([[1, '*', 3], [4, '*', 6], [7, '*', 9]]) == True
assert checkColumns([[1, 2, '*'], [4, 5, '*'], [7, 8, '*']]) == True

In [233]:
def checkRows(board):
    return any(checkWin(x) for x in board)

assert checkRows([[1, 2, 3], [4, 5, 6], [7, 8, 9]]) == False
assert checkRows([['*', '*', 3], [4, 5, 6], [7, 8, 9]]) == False
assert checkRows([[1, 2, 3], ['*', '*', '*'], [7, 8, 9]]) == True

In [234]:
def checkBoard(board):
    return checkRows(board) or checkColumns(board)

assert checkBoard([[1, 2, 3], [4, 5, 6], [7, 8, 9]]) == False
assert checkBoard([[1, 2, 3], ['*', '*', '*'], [7, 8, 9]]) == True
assert checkBoard([[1, '*', 3], [4, '*', 6], [7, '*', 9]]) == True

In [235]:
def sumBoard(board):
    size = len(board[0])
    return sum(board[y][x] 
                 for x in range(size)
                 for y in range(size)
                 if board[y][x] != '*')
    
assert sumBoard([[1, 2, 3], [4, 5, 6], [7, 8, 9]]) == 45

Return a list of boards that is deep copied. This is so the original data is not overwritten when numbers are removed.

In [236]:
def getBoards(input):
    boards = []
    for i in range(1, len(input)):
        boards.append([[x for x in row] for row in input[i]])

    return boards

getBoards([[[7, 4, 9, 5, 11, 17, 23, 2, 0, 14, 21, 24, 10, 16, 13, 6, 15, 25, 12, 22, 18, 20, 8, 19, 3, 26, 1]], [[22, 13, 17, 11, 0], [8, 2, 23, 4, 24], [21, 9, 14, 16, 7], [6, 10, 3, 18, 5], [1, 12, 20, 15, 19]]])

[[[22, 13, 17, 11, 0],
  [8, 2, 23, 4, 24],
  [21, 9, 14, 16, 7],
  [6, 10, 3, 18, 5],
  [1, 12, 20, 15, 19]]]

In [237]:
def printBoard(board):
    for row in board:
        print(' '.join(str.rjust(str(c),2) for c in row))
        
printBoard([[22, 13, 17, 11, 0],
  [8, 2, 23, 4, 24],
  [21, 9, 14, '*', 7],
  [6, 10, 3, 18, 5],
  [1, 12, 20, 15, 19]])

22 13 17 11  0
 8  2 23  4 24
21  9 14  *  7
 6 10  3 18  5
 1 12 20 15 19


## Solution 1

In [245]:
def solve_1(input):
    numbers, boards = input[0][0], getBoards(input)
        
    for i in numbers:
        for board in boards:
            for y in range(5):
                for x in range(5):
                    if board[y][x] == i:
                        board[y][x] = '*'
            if checkBoard(board):
                printBoard(board)
                return i * sumBoard(board)
    

solve_1(test_input)

 *  *  *  *  *
10 16 15  * 19
18  8  * 26 20
22  * 13  6  *
 *  * 12  3  *


4512

In [246]:
f"Solution 1: {solve_1(final_input)}"

64 65  6 86 53
10 56  2 88  *
11  *  * 84  *
 *  *  *  *  *
 *  * 19 14  4


'Solution 1: 39902'

## Solution 2

In [243]:
def solve_2(input):
    numbers, boards = input[0][0], getBoards(input)
        
    inplay = {i for i in range(len(boards))}
    for i in numbers:
        for boardNumber,board in enumerate(boards):
            for y in range(5):
                for x in range(5):
                    if board[y][x] == i:
                        board[y][x] = '*'
            if boardNumber in inplay and checkBoard(board):
                if len(inplay) == 1:
                    printBoard(board)
                    return i * sumBoard(board)
                inplay.remove(boardNumber)
    
solve_2(test_input)

 3 15  *  * 22
 * 18  *  *  *
19  8  * 25  *
20  *  *  *  *
 *  *  * 12  6


1924

In [242]:
f"Solution 2: {solve_2(final_input)}"

 *  *  *  * 32
 *  * 38  * 45
70  *  * 86  *
 *  *  * 66 60
 *  * 84  *  *


'Solution 2: 26936'